# Interactive PCE Visualization

Use the slider to step through iterations and see how PCE evolves.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from IPython.display import display

np.random.seed(42)

## Cost Function

Simple quadratic: $J(x) = (x - 5)^2$

In [2]:
def quadratic_cost(x: np.ndarray, x_star: float = 5.0) -> np.ndarray:
    """Simple quadratic bowl: J(x) = (x - x*)²"""
    return (x - x_star) ** 2

X_STAR = 5.0  # Optimal solution

## PCE Algorithm (with full history tracking)

In [3]:
def run_pce_with_history(
    cost_fn,
    mu_init: float = 0.0,
    sigma_init: float = 2.0,
    n_samples: int = 100,
    n_iterations: int = 50,
    elite_ratio: float = 0.1,
    alpha: float = 0.5,
    sigma_min: float = 0.01,
):
    """
    Run PCE and store full history for visualization.
    
    Returns:
        history: dict with all iteration data
    """
    mu = mu_init
    sigma = sigma_init
    n_elites = max(1, int(n_samples * elite_ratio))
    
    history = {
        'mu': [mu],
        'sigma': [sigma],
        'cost': [cost_fn(np.array([mu]))[0]],
        'samples': [],      # All samples at each iteration
        'sample_costs': [], # Costs of all samples
        'elite_indices': [],# Which samples were elites
        'elite_mean': [],   # Elite mean at each iteration
    }
    
    for iteration in range(n_iterations):
        # Step 1: Sample from current belief
        samples = mu + sigma * np.random.randn(n_samples)
        
        # Step 2: Evaluate costs
        costs = cost_fn(samples)
        
        # Step 3: Select elites (LOWEST cost = BEST)
        elite_indices = np.argsort(costs)[:n_elites]
        elite_samples = samples[elite_indices]
        
        # Step 4: Compute elite statistics
        elite_mean = np.mean(elite_samples)
        elite_std = np.std(elite_samples) + 1e-8
        
        # Store iteration data BEFORE update
        history['samples'].append(samples.copy())
        history['sample_costs'].append(costs.copy())
        history['elite_indices'].append(elite_indices.copy())
        history['elite_mean'].append(elite_mean)
        
        # Step 5: Update with EMA
        mu = (1 - alpha) * mu + alpha * elite_mean
        sigma = (1 - alpha) * sigma + alpha * elite_std
        sigma = max(sigma, sigma_min)
        
        # Store updated values
        history['mu'].append(mu)
        history['sigma'].append(sigma)
        history['cost'].append(cost_fn(np.array([mu]))[0])
    
    return history

## Run PCE

In [ ]:
# Run PCE and collect history
history = run_pce_with_history(
    cost_fn=quadratic_cost,
    mu_init=0.0,
    sigma_init=0.5,
    n_samples=100,
    n_iterations=50,
    elite_ratio=0.1,
    alpha=0.5,
)

print(f"PCE completed: {len(history['samples'])} iterations")
print(f"Initial cost: {history['cost'][0]:.4f}")
print(f"Final cost: {history['cost'][-1]:.6f}")

PCE completed: 50 iterations
Initial cost: 25.0000
Final cost: 0.000000


## Interactive Visualization

In [5]:
def plot_pce_iteration(iteration: int):
    """
    Plot PCE state at a given iteration.
    
    Left plot: Cost landscape with samples, elites, and mean trajectory
    Right plot: Cost convergence with current iteration marked
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # =====================================================================
    # Left: Cost landscape with samples
    # =====================================================================
    ax = axes[0]
    
    # Plot cost function
    x = np.linspace(-3, 10, 500)
    ax.plot(x, quadratic_cost(x), 'k-', lw=2, label='J(x)', zorder=1)
    
    # Plot optimal point
    ax.axvline(X_STAR, color='red', ls='--', lw=2, alpha=0.7, label=f'x* = {X_STAR}')
    
    # Get data for this iteration
    if iteration < len(history['samples']):
        samples = history['samples'][iteration]
        costs = history['sample_costs'][iteration]
        elite_idx = history['elite_indices'][iteration]
        elite_mean = history['elite_mean'][iteration]
        
        # Plot all samples (light green)
        ax.scatter(samples, costs, c='lightgreen', s=40, alpha=0.6, 
                   label='Samples', zorder=2, edgecolors='gray', linewidths=0.5)
        
        # Plot elite samples (dark green)
        elite_samples = samples[elite_idx]
        elite_costs = costs[elite_idx]
        ax.scatter(elite_samples, elite_costs, c='green', s=80, 
                   label='Elites', zorder=3, edgecolors='darkgreen', linewidths=1.5)
        
        # Plot elite mean
        ax.axvline(elite_mean, color='darkgreen', ls='-', lw=2, 
                   label=f'Elite mean = {elite_mean:.2f}', zorder=4)
    
    # Plot mean trajectory up to current iteration
    mu_history = history['mu'][:iteration+2]  # +2 because we have initial + updates
    cost_at_mu = [quadratic_cost(np.array([m]))[0] for m in mu_history]
    
    # Draw trajectory line
    ax.plot(mu_history, cost_at_mu, 'g-', lw=2, alpha=0.5, zorder=5)
    
    # Mark all previous means
    ax.scatter(mu_history[:-1], cost_at_mu[:-1], c='green', s=60, 
               alpha=0.4, zorder=5, marker='o')
    
    # Mark current mean (large)
    current_mu = history['mu'][iteration]
    current_cost = history['cost'][iteration]
    ax.scatter([current_mu], [current_cost], c='green', s=200, 
               zorder=6, marker='*', edgecolors='darkgreen', linewidths=2,
               label=f'Current μ = {current_mu:.2f}')
    
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('J(x)', fontsize=12)
    ax.set_title(f'PCE Iteration {iteration}', fontsize=14)
    ax.legend(loc='upper right', fontsize=9)
    ax.set_xlim(-3, 10)
    ax.set_ylim(-1, 30)
    ax.grid(True, alpha=0.3)
    
    # =====================================================================
    # Right: Cost convergence
    # =====================================================================
    ax = axes[1]
    
    # Plot full cost history
    all_costs = history['cost']
    iterations = np.arange(len(all_costs))
    
    ax.plot(iterations, all_costs, 'g-', lw=2, alpha=0.5, label='Cost J(μ)')
    ax.scatter(iterations, all_costs, c='green', s=30, alpha=0.3)
    
    # Mark current iteration
    ax.scatter([iteration], [history['cost'][iteration]], c='green', s=200, 
               zorder=10, marker='*', edgecolors='darkgreen', linewidths=2,
               label=f'Current: {history["cost"][iteration]:.4f}')
    ax.axvline(iteration, color='green', ls='--', lw=1, alpha=0.5)
    
    ax.set_xlabel('Iteration', fontsize=12)
    ax.set_ylabel('Cost J(μ)', fontsize=12)
    ax.set_title('Convergence', fontsize=14)
    ax.set_yscale('log')
    ax.set_xlim(-1, len(all_costs))
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [6]:
# Create interactive slider
interact(
    plot_pce_iteration,
    iteration=IntSlider(
        min=0, 
        max=len(history['samples'])-1, 
        step=1, 
        value=0,
        description='Iteration:',
        continuous_update=False,  # Only update when slider is released
        style={'description_width': 'initial'}
    )
);

interactive(children=(IntSlider(value=0, continuous_update=False, description='Iteration:', max=49, style=Slid…

## Animation (optional)

Run the cell below to see an automatic animation through all iterations.

In [7]:
from IPython.display import clear_output
import time

def animate_pce(delay: float = 0.3):
    """Animate through all PCE iterations."""
    for i in range(len(history['samples'])):
        clear_output(wait=True)
        plot_pce_iteration(i)
        time.sleep(delay)

# Uncomment to run animation:
# animate_pce(delay=0.2)